# Import Data

In [1]:
import sys

## Get the path dinamically to the src folder and add it to the path
sys.path.append("../src")

import importer

train_values, train_labels, test_values = importer.import_data(directory="../Data")

print(train_values.shape)
print(train_labels.shape)
print(test_values.shape)


(86868, 39)
(260601, 2)
(86868, 39)


# Impute Data

In [2]:
import imputator

train_values, train_labels, test_values = imputator.impute_data(train_values, train_labels, test_values)

# Encode Data

In [3]:
import encoder

train_values, train_labels, test_values = encoder.encode(train_values, train_labels, test_values)


## Geo encoder

# Train data Geo Values
geo1 = encoder.dummy_encode(train_values["geo_level_1_id"])
geo2 = encoder.dummy_encode(train_values["geo_level_2_id"])
geo3 = encoder.dummy_encode(train_values["geo_level_3_id"])

# Create a model

In [4]:
import model

model = model.neural_network(geo1.shape[1], geo2.shape[1], geo3.shape[1])

model.fit(geo3, [geo2, geo1], epochs=10, batch_size=32, validation_split=0.2)
model.save("model.h5")

Epoch 1/10


2023-10-05 16:17:49.953155: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2172/2172 [==============================] - 4s 2ms/step - loss: 0.2336 - dense_loss: 0.0670 - dense_1_loss: 0.1666 - val_loss: 0.1298 - val_dense_loss: 0.0062 - val_dense_1_loss: 0.1235
Epoch 2/10
2172/2172 [==============================] - 3s 1ms/step - loss: 0.1169 - dense_loss: 0.0058 - dense_1_loss: 0.1111 - val_loss: 0.0946 - val_dense_loss: 0.0056 - val_dense_1_loss: 0.0890
Epoch 3/10
2172/2172 [==============================] - 4s 2ms/step - loss: 0.0672 - dense_loss: 0.0053 - dense_1_loss: 0.0619 - val_loss: 0.0459 - val_dense_loss: 0.0050 - val_dense_1_loss: 0.0409
Epoch 4/10
2172/2172 [==============================] - 4s 2ms/step - loss: 0.0316 - dense_loss: 0.0047 - dense_1_loss: 0.0269 - val_loss: 0.0252 - val_dense_loss: 0.0044 - val_dense_1_loss: 0.0207
Epoch 5/10
2172/2172 [==============================] - 3s 1ms/step - loss: 0.0172 - dense_loss: 0.0042 - dense_1_loss: 0.0131 - val_loss: 0.0168 - val_dense_loss: 0.0040 - val_dense_1_loss: 0.0128
Epoch 6/10
2172/2172 

# Evaluate the model

In [5]:
import evaluator

evaluator.print_model_summary(model)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10213)]      0           []                               
                                                                                                  
 intermediate (Dense)           (None, 16)           163424      ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1364)         23188       ['intermediate[0][0]']           
                                                                                                  
 dense_1 (Dense)                (None, 31)           527         ['intermediate[0][0]']           
                                                                                              

# Submit

In [6]:
import submiter

submiter.submit(model, test_values)


Submission saved
